# main Stuff

In [12]:
MONTHS = {'01':"winter",
          '02':'winter',
          '03':"winter",
          '04':"spring",
          '05':'spring',
          '06':"spring",
          '07':'summer',
          '08':"summer",
          '09':"summer",
          '10':'autum',
          '11':'autum',
          '12': 'autum'} 

    #print(clusters.predict(array([5.9737, 52.4066])))

#testing "top10"
top10 = ["Polygonia c-album"]

if __name__ == "__main__":
    data = sc.textFile("/home/WUR/steen176/Downloads/butterflies.csv")
    corrected = data.map(lambda line: correctLine(line))
    for stagione in ['summer',"winter", 'autum', 'spring']:
        current_season = corrected.filter(lambda line: selectSeason(line))
    
        #turned of for testing purposes
        top10 = getRareSpecies(current_season)
        #print(top10)
        species = current_season.map(lambda x: x.split(',')[0]).collect()
        
        centers = {}
        for specie in species:
            current_specie = current_season.filter(lambda line: selectSpecies(line))
            centers[specie] = cluster(current_specie).clusterCenters
            
        #print(centers)    
        print(compareCenters(top10, centers))
        
      
        
        
    
            

KeyboardInterrupt: 

# Clustering
Methods including clustering and pre-processing 

In [2]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import SparseVector
from math import sqrt
NUM_CLUSTER = 4

def parseline(line):
    line = line.split(',')
    longti = float(line[8])
    latti = float(line[9])
    s_vec = SparseVector(2, [[0, longti], [1, latti]])
    return s_vec

def cluster(data):
    parsedData = data.map(lambda line: parseline(line))
    clusters = KMeans.train(parsedData, NUM_CLUSTER, maxIterations=20, runs=100, initializationMode="random")
    return clusters

def splitSeasons():
    pass
    
def error(point):
    center = clusters.centers[clusters.predict(point)]
    denseCenter = DenseVector(numpy.ndarray.tolist(center))
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

    WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
    print(WSSSE)

# Utility

In [3]:
def getSpeciesCount(rdd):
    pp.pprint(rdd[0])
    
def correctLine(line):
    line = line.split(',')
    if len(line) != 10:
        return ','.join(line[0:4] + [" ".join(line[4:6])]+ line[6:])
    else:
        return ','.join(line)
def selectSeason(line):
    line = line.split(',')
    month = line[6].split('-')[1]
    if MONTHS[month] == stagione:
        return True
    else:
        return False
    
def getRareSpecies(species):
    species_to_count = species.map(lambda x: x.split(',')[0])
    return countSpecies(species_to_count)
    
    
def selectSpecies(line):
    line = line.split(',')
    org = line[0]
    if specie == org:
        return True
    else:
        return False

def countSpecies(rdd):    
    pairs = rdd.map(lambda a: (a,1))
    counts = pairs.reduceByKey(lambda x,y: x+y)
    return counts.takeOrdered(10, key=lambda p: p[-1])

# Comparing clusters

In [10]:
import numpy
def centerdiffs(centerA, centerB, DIFF=0.1):
    longA = centerA[0]
    longB = centerB[0]
    lattA = centerA[1]
    lattB = centerB[1]
    
    longDiff = numpy.subtract(longA, longB)
    lattDiff = numpy.subtract(lattA, lattB)
    
    if longDiff < DIFF and lattDiff < DIFF:
        return True
    else:
        return False
    
def compareCenters(top10, centers):
    cohabit = []
    for rare_specie in top10:
        try:
            rest_keys = centers.keys()
            rest_keys.remove(rare_specie)
            rare_centers = centers[rare_specie]
            
            for rest_value in rest_keys:
                other = centers[rest_value]
                for center in rare_centers:
                    for oth in other:
                        if centerdiffs(center,oth):
                            cohabit.append([rare_specie, rest_value, tuple(oth.tolist())])
        except KeyError:
            continue
        except ValueError:
            pass
    return cohabit#[list(i) for i in set(map(tuple, cohabit))]

# Old stuff
We can ignore this but some parts might be usefull for development

In [ ]:
import pyspark
import pprint 
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel



pp = pprint.PrettyPrinter(indent=2)

def getRDD(fileName):
    rdd = sc.textFile(fileName)
    cols = rdd.map(lambda line: line.split(','))
    
    return cols

def getSpeciesCount(rdd):
    pp.pprint(rdd[0])
    
def countSpecies(rdd):
    pairs = rdd.map(lambda a: (a,1))
    counts = pairs.reduceByKey(lambda x,y: x+y)
    print(counts.takeOrdered(10, key=lambda p: p[-1]))
    
def getYears(rdd):
    dates = [x[6].split("-")[0] for x in rdd.collect() if len(x[6].split("-"))== 3 ]
    return list(set(dates))

def cluster(rdd):
    pass
    
if __name__ == "__main__":
    cols = getRDD('/home/WUR/steen176/Downloads/butterflies.csv')
    
    
    birds = cols.filter(lambda x: True if x[2] == u"1" else False)
    butterflies = cols.filter(lambda x: True if x[2] == u"4" else False)
    
    countSpecies(species)
    #year_list = getYears(cols)
    #print(year_list)
    